In [30]:
# !pip install opencv-python
# !pip install pillow

In [53]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
import torch as th 

import numpy as np
import cv2
from PIL import Image
import time
import pickle
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

In [54]:
class Cube:
    def __init__(self, size):
        self.size = size
        self.x = np.random.randint(0, self.size)
        self.y = np.random.randint(0, self.size)

    def __str__(self):
        return f'{self.x},{self.y}'

    def __sub__(self, other):
        return (self.x-other.x, self.y-other.y)

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def action(self, choise):
        if choise == 0:
            self.move(x=1, y=1)
        elif choise == 1:
            self.move(x=-1, y=1)
        elif choise == 2:
            self.move(x=1, y=-1)
        elif choise == 3:
            self.move(x=-1, y=-1)
        elif choise == 4:
            self.move(x=0, y=1)
        elif choise == 5:
            self.move(x=0, y=-1)
        elif choise == 6:
            self.move(x=1, y=0)
        elif choise == 7:
            self.move(x=-1, y=0)
        elif choise == 8:
            self.move(x=-0, y=0)

    def move(self, x=False, y=False):
        if not x:
            self.x += np.random.randint(-1, 2)
        else:
            self.x += x
        if not y:
            self.y += np.random.randint(-1, 2)
        else:
            self.y += y

        if self.x < 0:
            self.x = 0
        elif self.x >= self.size:
            self.x = self.size-1

        if self.y < 0:
            self.y = 0
        elif self.y >= self.size:
            self.y = self.size-1

In [55]:
class envCube(gym.Env):
    SIZE = 20
    OBSERVATION_SPACE_VALUES = (SIZE, SIZE, 3)
    ACTION_SPACE_VALUES = 9
    RETURN_IMAGE = False

    FOOD_REWARD = 25
    ENEMY_PENALITY = -300
    MOVE_PENALITY = -1

    d = {1: (255, 0, 0), 2: (0, 255, 0), 3: (0, 0, 255)}

    PLAYER_N = 1
    FOOD_N = 2
    ENEMY_N = 3

    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(self):
        super(envCube, self).__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(self.ACTION_SPACE_VALUES)
        # Example for using image as input (channel-first; channel-last also works):
        # self.observation_space = spaces.Box(
        #     low=0, high=255, shape=(N_CHANNELS, HEIGHT, WIDTH), dtype=np.uint8
        # )
        self.observation_space = spaces.Box(
            low=-self.SIZE + 1, high=self.SIZE - 1, shape=(4,), dtype=int
        )

    def reset(self, seed=None):
        self.player = Cube(self.SIZE)
        self.food = Cube(self.SIZE)
        while self.food == self.player:
            self.food = Cube(self.SIZE)

        self.enemy = Cube(self.SIZE)
        while self.enemy == self.player or self.enemy == self.food:
            self.enemy = Cube(self.SIZE)

        if self.RETURN_IMAGE:
            observation = np.array(self.get_image())
        else:
            observation = (self.player - self.food) + (self.player - self.enemy)
        observation = np.array(observation)
        self.episode_step = 0
        info = {}
        return observation, info

    def step(self, action):
        self.episode_step += 1
        self.player.action(action)
        self.food.move()
        self.enemy.move()

        if self.RETURN_IMAGE:
            new_observation = np.array(self.get_image())
        else:
            new_observation = (self.player - self.food) + (self.player - self.enemy)

        new_observation = np.array(new_observation)
        if self.player == self.food:
            reward = self.FOOD_REWARD
        elif self.player == self.enemy:
            reward = self.ENEMY_PENALITY
        else:
            reward = self.MOVE_PENALITY

        
        terminated = False
        if (
            self.player == self.food
            or self.player == self.enemy
            or self.episode_step >= 200
        ):
            terminated = True

        info = {}
        
        truncated = False
        return new_observation, reward, terminated, truncated, info

    def render(self):
        img = self.get_image()
        img = img.resize((400, 400))
        cv2.imshow("Predatoc", np.array(img))
        if (
            self.player == self.food
            or self.player == self.enemy
            or self.episode_step >= 200
        ):
            cv2.waitKey(1500)
        else:
            cv2.waitKey(200)

    def get_image(self):
        env = np.zeros((self.SIZE, self.SIZE, 3), dtype=np.uint8)
        env[self.food.x][self.food.y] = self.d[self.FOOD_N]
        env[self.player.x][self.player.y] = self.d[self.PLAYER_N]
        env[self.enemy.x][self.enemy.y] = self.d[self.ENEMY_N]
        img = Image.fromarray(env, "RGB")
        return img

In [56]:
env = envCube()
check_env(env)

In [39]:
# model = A2C(
#     "MlpPolicy",
#     env,
#     verbose=1,
#     tensorboard_log="./tb_logs",
#     learning_rate=5e-4,
#     # policy_kwargs={
#     #     "net_arch": [dict(vf=[32, 32], pi=[16, 16])],
#     #     "activation_fn": th.nn.ReLU,
#     # },
# )

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.policy

In [ ]:
# model.learn(
#     total_timesteps=int(1.5e6),
#     progress_bar=True,
#     tb_log_name="S20_A2C_Net_1M5",
# )
# model.save("S20_A2C_Net_1M5")

In [57]:
model = A2C.load("S20_A2C_Net_1M5", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [51]:
# env测试代码
eposides = 10
for ep in range(eposides):
    # 初始化
    obs, _ = env.reset()
    terminated = False
    rewards = 0
    while not terminated:
        # action = env.action_space.sample()
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, terminated, info, _ = env.step(action)
        env.render()
        rewards += reward
    print(terminated,rewards)
env.close()

True 9
True 13
True -5
True 19
True 6
True -311
True 13
True 1
True 8
True 3


In [61]:
mean_reward, std_reward = evaluate_policy(
    model, model.get_env(), deterministic=False, render=False, n_eval_episodes=10
)
print(mean_reward)

9.4


: 